In [1]:
#from nipype.interfaces import niftyreg
import numpy as np
from skimage import io
from pathlib import Path
import re
import ants
from skimage.transform import resize
from tqdm import tqdm
from skimage.morphology import skeletonize_3d, binary_dilation, binary_closing
from scipy.ndimage import distance_transform_edt
import tifffile as tif
from scipy.ndimage import binary_fill_holes 
import cc3d
from scipy.io import loadmat, savemat
import skan
import sknw
import networkx as nx
import pickle
import os

# Define connected componnet removal

In [2]:
def remove_small_comps_3d(image, thresh = 500):
    """
    

    Parameters
    ----------
    image : binary np array with uint8 elements
        3d numpy matrix, connected components will be removed form this image
    thresh : int64
        smallest connected components to keep

    Returns
    -------
    np.array with uint8 elements, binary
        binary image with connected components below the threshold removed.

    """
    img_lab, N = cc3d.connected_components(image,return_N=True)
    unique, counts = np.unique(img_lab, return_counts=True)
    unique_keep = unique[counts>thresh]
    unique_keep = np.delete(unique_keep,[0])
    img_filt = np.zeros(img_lab.shape).astype('int8')
    img_filt[np.isin(img_lab,unique_keep)] = 1
    return img_filt.astype('uint8')   

def fill_holes(img,thresh=100):
    #res = np.zeros(img.shape)
    for i in np.unique(img)[::-1]:
        _tmp = (img==i)*1.0
        _tmp = _tmp.astype('int8')
        _tmp = remove_small_comps_3d(_tmp,thresh=thresh)
        img[_tmp==1] = i
    res = img.astype('int8')
    return res

# Get mean predictions

In [3]:
directory = Path('matt_preds')
files  = directory.glob('*_mean.npy')
files = sorted([x.as_posix() for x in files])

In [4]:
len(files)

390

# High bias low varience segmentation

With removal of connected components under 500 pixels

In [5]:
min_prob = 0.75
max_var = 0.1
for file in tqdm(files[::-1]):
    if not os.path.exists(re.sub('mean','seg',file)):
        mean = np.load(file)
        std = np.load(re.sub('mean','std',file))
        seg = np.zeros(mean.shape[1:])
        seg[(mean[1,:,:,:] > min_prob) * (std[1,:,:,:] < max_var)] = 1
        seg[(mean[2,:,:,:] > min_prob) * (std[2,:,:,:] < max_var)] = 2
        seg = seg.astype('int8')
        seg = (seg==1)*1
        seg = fill_holes(seg)
        np.save(re.sub('mean','seg',file),seg)
        #savemat(re.sub('mean.npy','seg.mat',file),{'FinalImage':fill_holes(binary_dilation(binary_dilation(seg)))})
        #tif.imwrite(re.sub('mean.npy','seg.tif',file),seg)

100%|██████████| 390/390 [00:00<00:00, 658.10it/s]


# ANTs registration

## Upsample raw images

In [6]:
image_path = Path('../TBI')
images = list(image_path.glob('*_*/XYZres*?[0-9].tif'))
images = sorted([x.as_posix() for x in images])
len(images)

432

In [7]:
for i in tqdm(range(len(images))[:]):
    if not os.path.exists('matt_preds/'+re.sub('.tif','_upsamp.tif',Path(images[i]).name)):
        image = io.imread(images[i])
        image = np.swapaxes(image,0,1)
        image = np.swapaxes(image,1,3)
        image = resize(image,(2,507,507,252),preserve_range=True)
        zeros = np.zeros((1,507,507,252)).astype('uint16')
        image = np.append(image,zeros,axis = 0)
        image = image.astype('float16')
        io.imsave('matt_preds/'+re.sub('.tif','_upsamp.tif',Path(images[i]).name),image)

100%|██████████| 432/432 [00:00<00:00, 469.37it/s]


## reshape numpy seg files and resave as tif

In [8]:
directory_seg = Path('matt_preds')
files_seg  = list(directory_seg.glob('*_seg.npy'))
files_seg = sorted([x.as_posix() for x in files_seg])

In [9]:
for i in tqdm(range(len(files_seg))):
    if not os.path.exists(re.sub('.npy','.tif',files_seg[0])):
        tmp = np.load(files_seg[i])
        tmp = np.reshape(tmp,(1,507,507,252))
        io.imsave(re.sub('.npy','.tif',files_seg[0]),tmp)

100%|██████████| 390/390 [00:00<00:00, 64025.15it/s]


## Register images and transform masks

In [10]:
directory_seg = Path('matt_preds')
images = list(directory_seg.glob('*_0001_upsamp.tif'))
images = sorted([x.as_posix() for x in images])[::-1]
images[0]

'matt_preds/XYZres99_0001_upsamp.tif'

In [11]:
for i in tqdm(range(len(images))):
    if os.path.exists(re.sub('upsamp','seg',images[i])):
        if not os.path.exists(re.sub('seg','seg_warped',re.sub('upsamp','seg',re.sub('_0001','',images[i])))):
            fix = ants.image_read(re.sub('_0001','',images[i]))
            mov = ants.image_read(images[i])
            fix_mask = ants.image_read(re.sub('upsamp','seg',re.sub('_0001','',images[i])))
            mov_mask = ants.image_read(re.sub('upsamp','seg',images[i]))
            mytx = ants.registration(fixed = fix,
                                    moving = mov,
                                    type_of_transform = 'Rigid'
                                    )
            warpedmask = ants.apply_transforms(fixed = fix_mask,
                                               moving = mov_mask,
                                               transformlist = mytx['fwdtransforms'],
                                               interpolator = 'nearestNeighbor'
                                              )
            io.imsave(re.sub('seg','seg_warped',re.sub('upsamp','seg',images[i])),warpedmask.numpy())
            io.imsave(re.sub('seg','seg_warped',re.sub('upsamp','seg',re.sub('_0001','',images[i]))),fix_mask.numpy())
            #print(2 * np.sum(warpedmask.numpy()*fix_mask.numpy())/(np.sum(warpedmask.numpy())+np.sum(fix_mask.numpy())))

  0%|          | 0/200 [00:00<?, ?it/s]/tmp/ipykernel_2695885/3743514629.py:17: UserWarning:

matt_preds/XYZres99_0001_seg_warped.tif is a low contrast image

/tmp/ipykernel_2695885/3743514629.py:18: UserWarning:

matt_preds/XYZres99_seg_warped.tif is a low contrast image

  0%|          | 1/200 [02:56<9:45:43, 176.60s/it]/tmp/ipykernel_2695885/3743514629.py:17: UserWarning:

matt_preds/XYZres98_0001_seg_warped.tif is a low contrast image

/tmp/ipykernel_2695885/3743514629.py:18: UserWarning:

matt_preds/XYZres98_seg_warped.tif is a low contrast image

  2%|▎         | 5/200 [15:01<9:49:46, 181.47s/it]/tmp/ipykernel_2695885/3743514629.py:17: UserWarning:

matt_preds/XYZres94_0001_seg_warped.tif is a low contrast image

/tmp/ipykernel_2695885/3743514629.py:18: UserWarning:

matt_preds/XYZres94_seg_warped.tif is a low contrast image

  4%|▍         | 8/200 [24:00<9:34:36, 179.56s/it]/tmp/ipykernel_2695885/3743514629.py:17: UserWarning:

matt_preds/XYZres91_0001_seg_warped.tif is a low co

KeyboardInterrupt: 

## Save Matlab .mat file of registered images

In [12]:
directory_seg = Path('matt_preds')
images = list(directory_seg.glob('*_0001_seg_warped.tif'))
images = sorted([x.as_posix() for x in images])[::-1]

In [13]:
for image in tqdm(images):
    #if not os.path.exists(re.sub('_seg_warped.tif','_seg_warped_single.mat',re.sub('_0001','',image))):
        img_0001 = io.imread(image)
        img = io.imread(re.sub('_0001','',image))
        seg = img*img_0001
        seg = (seg==1)*1
        seg = seg.astype('int8')
        seg = fill_holes(seg)
        savemat(re.sub('_seg_warped.tif','_seg_warped_single.mat',re.sub('_0001','',image)),{'FinalImage':seg})

 64%|██████▍   | 125/195 [14:24<08:04,  6.92s/it]


KeyboardInterrupt: 